In [3]:
#%pip install torch --index-url https://download.pytorch.org/whl/cpu
%pip install datasets python-dotenv
from utils import *


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [29]:

import random
from datasets import load_dataset, Dataset
import json
from openai.types.chat.chat_completion import ChatCompletion
import os


default_or_options = {
    "max_tokens": 8191,

    "extra_body": {
        "provider": {
            "ignore": [
                "Hyperbolic"
            ],
            "require_parameters": True,
        },
    }
}
domain_events = load_dataset("lucaelin/generic_domain_events_v1", split="train")


def generate_event_dialog(entry):
    results = []

    available_models = [
        #'anthropic/claude-3.5-sonnet-20240620',
        #'openai/chatgpt-4o-latest',
        #'openai/gpt-4o-mini-2024-07-18',
        #'meta-llama/llama-3.1-405b-instruct',
        #'meta-llama/llama-3.1-70b-instruct',
        #'meta-llama/llama-3.1-70b-instruct',
        "mistralai/mistral-nemo",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        "mistralai/mixtral-8x22b-instruct",
        #"mistralai/mixtral-8x22b-instruct",
        #"mistralai/mistral-small",
        "mistralai/mistral-large"
    ]

    related_entries = domain_events.filter(lambda x: x["profession"] == entry["profession"] and x["process"] == entry["process"]).to_list()


    model = random.choice(available_models)

    select_event_name = [{
        "type": "function",
        "function": {
            "name": "select_event_name",
            "description": "Select an event name that continues the currently ongoing process.",
            "parameters": {
                "type": "object",
                "properties": {
                    "event_name": {
                        "type": "string",
                        "enum": [event["name"].strip() for event in related_entries],
                    },
                },
                "required": ["event_name"],
            },
        },
    }]

    generate_event_functions = [
        {
            "type": "function",
            "function": {
                "name": f"generate_{event["name"].replace(' ','_').lower()}_event",
                "description": event["name"].strip()+'\n'+event["description"],
                "parameters": json.loads(event['schema']),
            },
        } for event in related_entries
    ]

    events = []

    while len(events) < random.randint(2, 6):
        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"You roleplay an expert {entry['profession']} currently in the process of {entry['process']}. Be creative during this roleplay, you are in the lead, so you need to fill the story with detail.\n"+
                    "Your task is to note down the progress made. You should be creative and fill the story with detail."
            }, {
                "role": "user",
                "content": entry['situation']+'\n'+
                    f"Please select one of the available events to describes the currently ongoing process of {entry['process']}. Use the select_event_name function to select an event name.",
            }]+ ([{
                "role": "assistant",
                "content": '\n'.join([event["name"] for event in events]),
            }, {
                "role": "user",
                "content": f"Please choose a different event to continue the current process. Use the select_event_name function to select an event name.",
            }] if events else []),
            tools=select_event_name,
            tool_choice="required",
            **default_or_options,
        )
        print(openrouter_response)
        event_name_selection = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
        event_name = event_name_selection["event_name"].strip()
        print('selected', event_name)

        selected_event_functions = [function for function in generate_event_functions if function["function"]["description"].split('\n')[0].strip() == event_name]
        print('available functions', [function["function"]["description"].split('\n')[0].strip() for function in generate_event_functions])
        print('selected function', selected_event_functions)

        openrouter_response: ChatCompletion = openrouter.chat.completions.create(
            model=model,
            messages=[{
                "role": "system",
                "content": f"You roleplay an expert {entry['profession']} currently in the process of {entry['process']}. Be creative during this roleplay, you are in the lead, so you need to fill the story with detail.\n"+
                    "Your task is to note down the progress made. You should be creative and fill the story with detail."
            }, {
                "role": "user",
                "content": entry['situation']+'\n'+
                    f"Please select one of the available events to describes the currently ongoing process of {entry['process']}. Use the select_event_name function to select an event name.",
            }]+ ([{
                "role": "assistant",
                "content": '\n'.join([json.dumps(event) for event in events]),
            }, {
                "role": "user",
                "content": f"Please continue this process using the {event_name} event. Use the generate_event function to generate the event.",
            }] if events else []),
            tools=selected_event_functions,
            tool_choice="required",
            **default_or_options,
        )
        event = json.loads(openrouter_response.choices[0].message.tool_calls[0].function.arguments)
        print(event_name, ':', event)

        #print(request)
        events.append({"name":event_name, 'content': event})

    openrouter_response: ChatCompletion = openrouter.chat.completions.create(
        model=model,
        messages=[{
            "role": "system",
            "content": f"You are the digital Assistant of an expert {entry['profession']} currently accompanying the process of {entry['process']}.\n"+
                "Your task is to provide situational commentary on the events that have unfolded.\n"+
                "Here is an example from a Dentist roleplay:\n"+
                " - 'Adam has 3 cavities, 2 of which are in the back of his mouth. He has a toothache and is in pain.'\n"+
                " - 'The waiting room is full of patients, and the receptionist is on the phone with a supplier while we are preparing the x-ray on Thomas Delvenstein.'\n"+
                " - 'The dental hygienist is cleaning the tools and preparing the room for the next patient and the dentist is in the middle of a procedure.'\n"+
                "You should be creative, but never invent new information, only summarize the events that have unfolded into a single sentence. You should keep your commentary concise. Do not mention the protagonist by name, refer to them as 'you'.",
        }, {
            "role": "user",
            "content": entry['situation']+'\n'+
                f"Please provide a single sentence commentary on these events:\n"+
                '\n'.join([json.dumps(event) for event in events]),
        }],
        **default_or_options,
    )
    comment = openrouter_response.choices[0].message.content
    print(comment)

    results.append({
        "profession": entry["profession"],
        "process": entry["process"],
        "situation": entry["situation"],
        "situation_json": entry["situation_json"],
        "events": events,
        "comment": comment,
    })
    
    return results


situations = load_dataset("lucaelin/domain_situations_v1", split="train[0:10]")
situational_event_commentary = situations.map(
    sane_batch_map(generate_event_dialog, {"profession": [], "process": [], "situation": [], "situation_json": [], "events": [], "comment": []}), 
    batched=True, 
    batch_size=1, 
    num_proc=1, 
    remove_columns=situations.column_names
)
situational_event_commentary.sort(["profession", "process"]).to_list()
#situational_event_commentary.sort(["profession", "process"]).push_to_hub("lucaelin/generic_situational_event_commentary_v1")

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

ChatCompletion(id='gen-1729979000-pRy6XPcheZy4qBmIRpHy', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='h4HklZNoR', function=Function(arguments='{"event_name": "Invoice generated"}', name='select_event_name'), type='function', index=0)]))], created=1729979000, model='mistralai/mistral-large', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=500, total_tokens=527, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')
selected Invoice generated
available functions ['Account aged', 'Account marked for collection', 'Customer credit check passed', 'Customer credit limit exceeded', 'Customer disputes charge', 'Debt written off', 'Invoice adjusted', 'Invoice generated', 'Payment plan set up', 'Payment received']
sele

Map:  10%|█         | 1/10 [00:24<03:40, 24.55s/ examples]

In the midst of the Accounts Receivable process, you have generated invoice INV-12345 for John Doe, due by October 31st, while also noting that John Doe has exceeded his credit limit.
ChatCompletion(id='gen-1729979024-w60OBdNPdkhpfZ7nNmoa', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='5D3VQVBMq', function=Function(arguments='{"event_name": "Budget approved"}', name='select_event_name'), type='function', index=0)]))], created=1729979024, model='mistralai/mistral-large', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=28, prompt_tokens=426, total_tokens=454, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')
selected Budget approved
available functions ['Budget approved', 'Budget closed', 'Budget created', 'Budget forecast'

Map:  20%|██        | 2/10 [01:04<04:30, 33.80s/ examples]

...

Map:  90%|█████████ | 9/10 [02:20<00:12, 12.30s/ examples]

 As you complete formatting the Quarterly Financial Report and Jane Doe finishes proofreading it, you ensure that the document is ready for final review, polishing the last details to maintain the high standards of your work.
ChatCompletion(id='gen-1729979140-BZGy4PCiKK4CGeoNA6FG', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='juQCxgX0S', function=Function(arguments='{"event_name": "Email Received"}', name='select_event_name'), type='function', index=0)]))], created=1729979140, model='mistralai/mixtral-8x22b-instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=514, total_tokens=541, completion_tokens_details=None, prompt_tokens_details=None), provider='Mistral')
selected Email Received
available functions ['Email Archived', 'Ema

Map: 100%|██████████| 10/10 [02:26<00:00, 14.65s/ examples]

 Amidst the flurry of emails, you promptly respond to John Doe's meeting request, confirming your availability and requesting an agenda.


[{'profession': 'Accountant',
  'process': 'Accounts Receivable',
  'situation': 'In a bustling yet quiet office, an Accountant sits at a sturdy mahogany desk, immersed in the Accounts Receivable process. They are surrounded by a plethora of tools and equipment, all essential to their role. On their right, a reliable laptop - its silver exterior glinting under the warm office lights - is open, displaying the latest version of a sophisticated accounting software, its colorful pie charts and graphs a testament to its efficiency. To their left, a calculator sits alongside a high-quality printer, both in pristine condition. The accountant is not alone; their colleagues from the finance team are scattered around the room, each absorbed in their own tasks, the soft tapping of keys providing a rhythmic backdrop to the office. As the accountant ensures that customers pay their outstanding invoices on time and maintains accurate financial records, they are also in close communication with the f